# Segmenting and Clustering Neighborhoods in Toronto

_The first task requires to scrape and parse the given web-page. I'm going to use BeautifulSoup4 and requests libraries for this purpose._

In [51]:
# using requests load given web-page into object page 
import requests
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page

<Response [200]>

_Also I need pandas for loading parsed data into dataframe_

In [52]:
import pandas as pd

_Let's install BeautifulSoup4_

In [53]:
!pip install beautifulsoup4

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [ ]:
!conda install -c anaconda beautifulsoup4

In [54]:
#here we parse page content
page.content
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [65]:
#print(soup.prettify())

_Let's create dataframe for our data. It consists of 3 columns PostalCode, Borough, Neighborhood. And than load data into it from object soup_ 

In [57]:
#Let's create dataframe for our data. It consists of 3 columns  
CACodeDf = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

In [63]:
#And finaly, let's load data from our page into dataframe  CACodeDf
i = 0
for tr in soup.find_all('tr',  text=True)[1:]:
    tds = tr.find_all('td')  
    my_str = tds[2].text.replace('\n', '')
    CACodeDf.loc[i] = [tds[0].text]+ [tds[1].text] + [my_str]
    i = i+1
    #print(tds[0].text)
print      ("done")

done


_Here is the result. First 10 rows of our dataset_

In [64]:
CACodeDf.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [66]:
CACodeDf.shape

(288, 3)

_Now I have to process the data accordingly instructions given into assignment. First, I should remove all rows where Borough is Not assigned_

In [67]:
#Now , since we have dataframe filled with all necessary data, we should clear it from inappropriate rows
CACodeDf = CACodeDf[CACodeDf.Borough != "Not assigned"]
CACodeDf.head(10) 

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [68]:
CACodeDf.shape

(211, 3)

_Now, let's update values of the cells where  Neighborhood is Not assigned. The new value of Neighborhood  for such rows must be equal its Borough value_

In [69]:
CACodeDf.loc[CACodeDf['Neighborhood'] == "Not assigned", ['Neighborhood']] = CACodeDf['Borough']
CACodeDf.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [70]:
CACodeDf.shape

(211, 3)

_Finally, I have to transform our dataframe into the new one, so Postal Code would be unique_

In [71]:
df = CACodeDf[['PostalCode','Borough']]
#df.drop_duplicates()
df=df.drop_duplicates(['PostalCode','Borough'])[['PostalCode','Borough']]
df['Neighborhood'] = df.apply(lambda row: "", axis=1)
df.head(10)
#df.shape

,PostalCode,Borough,Neighborhood
2,M3A,North York,
3,M4A,North York,
4,M5A,Downtown Toronto,
6,M6A,North York,
8,M7A,Queen's Park,
10,M9A,Etobicoke,
11,M1B,Scarborough,
14,M3B,North York,
15,M4B,East York,
17,M5B,Downtown Toronto,


In [72]:
for index, row in df.iterrows():
    my_str = ''
    for index2, row2 in CACodeDf.iterrows():
        if row2['PostalCode'] == row['PostalCode'] and row2['Borough'] == row['Borough']:
            my_str = row2['Neighborhood']+","+my_str
    row['Neighborhood'] = my_str[:-1]
df.head(10)     

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
6,M6A,North York,"Lawrence Manor,Lawrence Heights"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Malvern,Rouge"
14,M3B,North York,Don Mills North
15,M4B,East York,"Parkview Hill,Woodbine Gardens"
17,M5B,Downtown Toronto,"Garden District,Ryerson"


In [73]:
df.shape

(103, 3)

_So, you can see the final dataset consists of 103 rows.<br> The next step is defining location for each Borough. I prefer using the csv of Geospatial data. And the line below imports the csv into my project_

In [ ]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

_Now let's load location data into the new dataftame  dfcoord_

In [41]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


_Let's increase our dataframe df with 2 columns Latitude and Longitude and assign its values from the dataframe dfcoord shown above_

In [44]:
df['Latitude'] = df.apply(lambda row: "", axis=1)
df['Longitude'] = df.apply(lambda row: "", axis=1)
df.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,,
3,M4A,North York,Victoria Village,,
4,M5A,Downtown Toronto,"Regent Park,Harbourfront",,
6,M6A,North York,"Lawrence Manor,Lawrence Heights",,
8,M7A,Queen's Park,Queen's Park,,


In [51]:
for index, row in df.iterrows():
    for index2, row2 in df_coord.iterrows():
        if row2['Postal Code'] == row['PostalCode'] :            
            row['Latitude'] = row2['Latitude']
            row['Longitude'] = row2['Longitude']
#df.reset_index(inplace=True)
df.drop('index',axis=1, inplace=True)
df.head(10)    

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
5,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
6,M1B,Scarborough,"Malvern,Rouge",43.8067,-79.1944
7,M3B,North York,Don Mills North,43.7459,-79.3522
8,M4B,East York,"Parkview Hill,Woodbine Gardens",43.7064,-79.3099
9,M5B,Downtown Toronto,"Garden District,Ryerson",43.6572,-79.3789


_Here is the final dataset ready for further exploration_